# Heidelberg Tutorial

In this tutorial we will be learning how to use NeuroTorch to train a neural network to recognize audio recordings of spoken digits.

## Setup

You can install the dependencies by running the following commands:

In [ ]:
!pip install -r heidelberg_requirements.txt
!pip install git+https://github.com/JeremieGince/NeuroTorch.git

If you have a cuda device and want to use it for this tutorial (it is recommended to do so), you can uninstall pytorch with `pip uninstall torch` and re-install it with the right cuda version by generating a command with [PyTorch GetStarted](https://pytorch.org/get-started/locally/) web page.

After setting up the virtual environment, we will need to import the necessary packages.

In [1]:
import os
import pprint
from collections import OrderedDict

import torch
from pythonbasictools.device import log_device_setup, DeepLib
from pythonbasictools.logging import logs_file_setup

from applications.heidelberg.dataset import get_dataloaders
import neurotorch as nt
from neurotorch import Dimension, DimensionProperty
from neurotorch.modules import HeavisideSigmoidApprox
from neurotorch.callbacks import CheckpointManager, LoadCheckpointMode
from neurotorch.metrics import ClassificationMetrics
from neurotorch.modules import SequentialModel
from neurotorch.modules.layers import SpyLIFLayer, SpyLILayer
from neurotorch.trainers import ClassificationTrainer
from neurotorch.regularization import RegularizationList, L2, L1

In [ ]:
logs_file_setup("heidelberg_tutoriel", add_stdout=False)
log_device_setup(deepLib=DeepLib.Pytorch)
torch.cuda.set_per_process_memory_fraction(0.5)

## Initialization

Here we're initializing a callback of the trainer used to save the network during the training.

In [ ]:
checkpoint_folder = f"./checkpoints_2HH1024"
os.makedirs(checkpoint_folder, exist_ok=True)
checkpoint_manager = CheckpointManager(checkpoint_folder, metric="val_accuracy", minimise_metric=False)

It's the time to defined our training and networks parameters. In the following,

- the number of iteration is the number of time the trainer will pass through the entire training dataset;
- the batch size is the number of samples that will be loaded at the same time for each forward and backward pass;
- the learning rate is the learning rate used by the optimizer;
- the random seed is the seed used by the random number generator;
- the number of steps is the number of euler integration steps performed by the model during each forward and backward pass;
- the number of hidden neurons is the number of neurons that will be used in the hidden layer of the model;
- dt is the time step of the euler integration;

In [ ]:
# Training parameters
n_iterations = 100
batch_size = 256  # Increase this if you have a GPU with more memory and decrease it otherwise.
learning_rate = 2e-4
seed = 42

# Network parameters
n_steps = 100
n_hidden_neurons = 1024
dt = 1e-3

In [ ]:
torch.manual_seed(seed)


Here is an exemple of the Heidelberg dataset:
![Heidelberg_exemples](images/heidelberg_exemples.png)

In [ ]:
dataloaders = get_dataloaders(
	batch_size=batch_size,
	train_val_split_ratio=0.95,
)

We can finally define our model. The sequential model is a neural network that is composed of a list of layers. The list of layers is ordered and the first layer is the input layer. The last layer is the output layer. The sequential model can be found in the subpackage `neurotorch.modules`. Note that we can specify the input transform that will be applied to the input data before the forward pass through the list of layers. If the transformation is computationally expensive, it is recommended to put it in the dataloaders instead of the model for the training. Usually the input transform in the model is used when the model is used for inference.

In [ ]:
network = SequentialModel(
	layers=[
		SpyLIFLayer(
			input_size=nt.Size([
				Dimension(None, DimensionProperty.TIME),
				Dimension(dataloaders["test"].dataset.n_units, DimensionProperty.NONE)
			]),
			output_size=n_hidden_neurons,
			use_recurrent_connection=True,
			spike_func=HeavisideSigmoidApprox,
			dt=dt,
			# spikes_regularization_factor=1e-6,
		),
		SpyLIFLayer(
			input_size=n_hidden_neurons,
			output_size=n_hidden_neurons,
			use_recurrent_connection=True,
			dt=dt,
			# spikes_regularization_factor=1e-6,
		),
		SpyLILayer(dt=dt, output_size=dataloaders["test"].dataset.n_classes),
	],
	name=f"heidelberg_network",
	checkpoint_folder=checkpoint_folder,
)

After the instantiation of the model, we have to build it, so it can infer the missing sizes of the layers in the model and create the tensors that will be used during the training.

In [ ]:
network.build()
print(network)

In [ ]:
regularization = RegularizationList([
	L2(network.parameters(), Lambda=1e-5),
	L1(network.parameters(), Lambda=1e-6),
])

## Training

In the next cell, we create the trainer that will be used to trained our network. They are several types of trainers that can be found in the subpackage `neurotorch.trainers` like the `ClassificationTrainer`, the `RegressionTrainer` or the `PPOTrainer`. In this tutorial, we will use the `ClassificationTrainer` because we are trying to classify some images. Note that the callbacks are some object that will be called during the training. They are found in the subpackage `neurotorch.callbacks` like the `CheckpointManager`.

In [ ]:
trainer = ClassificationTrainer(
	model=network,
	# regularization=regularization,
	optimizer=torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0.0),
	# regularization_optimizer=torch.optim.SGD(regularization.parameters(), lr=1e-2, weight_decay=0.0),
	callbacks=checkpoint_manager,
	verbose=True,
)

The training will start! Let's see how our simple network will perform on this task.

Note: the argument `force_overwrite` is used to overwrite the existing checkpoint if it exists and the argument `load_checkpoint_mode` if the model will be loaded from the last checkpoint (`LoadCheckpointMode.LAST_ITR`) or from the best checkpoint (`LoadCheckpointMode.BEST_ITR`). The loading mode is really useful when you want to stop the training and restart it later.

In [ ]:
training_history = trainer.train(
	dataloaders["train"],
	dataloaders["val"],
	n_iterations=n_iterations,
	load_checkpoint_mode=LoadCheckpointMode.LAST_ITR,
	# force_overwrite=True,
)

KeyboardInterrupt: 

The training is finished! Let's see how our network learned over the iterations.

Note: the next graphs are really useful to see if the model overfit or underfit the data over the iterations. It's why you can use the callback `TrainingHistoryVisualizationCallback` to see those graphs in real time.

In [ ]:
training_history.plot(show=True)

We can load the model at the best or the last iteration for the testing phase.

In [ ]:
network.load_checkpoint(checkpoint_manager.checkpoints_meta_path, LoadCheckpointMode.BEST_ITR, verbose=True)

We can now see how our network perform on the test dataset.

In [ ]:
predictions = {
	k: ClassificationMetrics.compute_y_true_y_pred(network, dataloader, verbose=True, desc=f"{k} predictions")
	for k, dataloader in dataloaders.items()
}

In [ ]:
accuracies = {
	k: ClassificationMetrics.accuracy(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}
pprint.pprint(accuracies)

Let's see the results on other popular classification metrics.

In [ ]:
precisions = {
	k: ClassificationMetrics.precision(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}
recalls = {
	k: ClassificationMetrics.recall(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}
f1s = {
	k: ClassificationMetrics.f1(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}

In [ ]:
results = OrderedDict(dict(
	network=network,
	accuracies=accuracies,
	precisions=precisions,
	recalls=recalls,
	f1s=f1s,
))

In [ ]:
pprint.pprint(results, indent=4)

This is the end of the tutorial!

# Conclusion

NeuroTorch is a library that allows you to easily build neural networks and train them. Moreover, it allows you to do audio signal classification using dynamics from neurosciences.